# Coleta de Dados WeRateDoga

In [ ]:
import pandas as pd
import requests
import tweepy
import json
import numpy as np
from bs4 import BeautifulSoup
from operator import itemgetter
import seaborn as sns
import matplotlib.pyplot as plt

## Config tweepy

In [ ]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

## Get twitter-archive-enhanced

In [ ]:
df_tweets =  pd.read_csv("../Data/twitter-archive-enhanced.csv")

## Get image_predictions

In [ ]:
url = " https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
r = requests.get(url)
with open("../Data/image_predictions.tsv", "wb") as code:
     code.write(r.content)

In [ ]:
df_images_predictions = pd.read_csv("../Data/image_predictions.tsv",sep='\t')

## Using tweepy for get tweet data

In [ ]:
data_twitter = []
for code in df_images_predictions.tweet_id:
    try:
        status_data = api.get_status(code,tweet_mode='extended')
        data_twitter.append(status_data._json)
        print(str(code)+" - encontrado")
    except:
        print(str(code)+" - não encontrado")

## Save downloaded jsons

In [ ]:
with open("../Data/tweet_json.txt", "w") as file:
    json.dump(data_twitter,file)


## Load downloaded jsons

In [ ]:
with open("../Data/tweet_json.txt") as tweet_json:  
    data_tweets_json = json.load(tweet_json)

## Creating dictionary downloaded data with tweepy

In [ ]:
tweets_data = []
key = 0
for data in data_tweets_json:
    tweets_data.append({}) 
    tweets_data[key]['id'] = data['id']
    tweets_data[key]['favorite_count'] = data['favorite_count']
    tweets_data[key]['retweet_count'] = data['retweet_count']
    key = key+1

## Converting dictionary to dataframe

In [ ]:
df_tweets_data = pd.DataFrame(tweets_data)

## Looking for trouble

In [ ]:
df_tweets

In [ ]:
df_tweets_data

In [ ]:
df_images_predictions

In [ ]:
df_tweets.name.value_counts()

In [ ]:
df_tweets.info()

In [ ]:
df_tweets_data.info()

In [ ]:
df_images_predictions.info()

In [ ]:
list(df_tweets.expanded_urls)

In [ ]:
list(df_tweets.text)

#### Quality
#### `df_tweets` dataframe
- Na coluna name, a falta de informação está sendo preechida com outros valores além de None("the", "a")
- As colunas retweeted_status_id,retweeted_status_user_id,in_reply_to_status_id e in_reply_to_user_id estão com o tipo float64 causando a perda de informação do valor do id
- A coluna source está trazendo toda a informação de uma div
- Os valores nulos da coluna expanded_urls estão sendo representados por nan
- Na coluna expanded_urls o mesmo url está sendo repetido várias vezes em um mesmo registro
- Existem tweets em que o texto começa com RT idicando um retweet
- A coluna timestamp não está representando um valor timestamp

#### `df_tweets_data` dataframe
- Registro faltantes(1457 de 2356)
#### Tidiness
- As colunas retweeted_status_id,retweeted_status_user_id e retweeted_status_timestamp do df_tweets não são úteis já que não  queremos retweets.
- Os dados do dataframe df_tweets_data fazem parte do df_tweets
- As colunas doggo, floofer, pupper e puppo respresentam a mesma informação que é o estagio do cão.
- O data frame `df_images_predictions` deve ser uma coluna no df_tweets indicando a raça do  cão caso seja possível

#### `df_tweets`: Na coluna nome, a falta de informação está sendo preechida com outros valores além de None("the", "a","an")

#### Define
Todos os nomes que foram pegos de forma errada não começam com letra maiúscula, então basta retirar estes e substituir por None.

#### Code

In [ ]:
df_tweets_clean = df_tweets.copy()
for key,row in df_tweets_clean.iterrows():
    if(row['name'][0].isupper() == False):
        df_tweets_clean['name'][key] = 'None'

#### Test

In [ ]:
df_tweets_clean.name.value_counts()

#### `df_tweets`: As colunas retweeted_status_id,retweeted_status_user_id,in_reply_to_status_id e in_reply_to_user_id estão com o tipo float64 causando a perda de informação do valor do id

#### Define
alterar o tipo das colunas usando a função .astype() do pandas tomando cuidado com os valores nulo, substituindo estes por -1

#### Code

In [ ]:
df_tweets_clean.retweeted_status_id = df_tweets_clean.retweeted_status_id.fillna(-1).astype(np.int64,copy=False)
df_tweets_clean.retweeted_status_user_id = df_tweets_clean.retweeted_status_user_id.fillna(-1).astype(np.int64,copy=False)
df_tweets_clean.in_reply_to_status_id = df_tweets_clean.in_reply_to_status_id.fillna(-1).astype(np.int64,copy=False)
df_tweets_clean.in_reply_to_user_id = df_tweets_clean.in_reply_to_user_id.fillna(-1).astype(np.int64,copy=False)

#### Test

In [ ]:
df_tweets_clean.info()

In [ ]:
df_tweets_clean.in_reply_to_status_id.value_counts()

In [ ]:
df_tweets_clean.in_reply_to_user_id.value_counts()

In [ ]:
df_tweets_clean.retweeted_status_id.value_counts()

In [ ]:
df_tweets_clean.retweeted_status_user_id.value_counts()

#### `df_tweets`: A coluna source está trazendo toda a informação de uma div

#### Define
percorrer todas as linhas e usar o BeautifulSoup para pegar a informação dentro da div

#### Code

In [ ]:
sources = []
for source in df_tweets_clean.source:
    bs = BeautifulSoup(source,'lxml')
    sources.append((bs.find('a').contents[0]))
sources
df_tweets_clean.source = sources

#### Test

In [ ]:
df_tweets_clean.info()

#### `df_tweets`: Os valores nulos da coluna expanded_urls estão sendo representados por nan

#### Define
usar a função fillna para alterar os valores NaN para "None"

#### Code

In [ ]:
df_tweets_clean.expanded_urls = df_tweets_clean.expanded_urls.fillna('None')

#### Test

In [ ]:
df_tweets_clean.info()

#### `df_tweets`: Na coluna expanded_urls o mesmo url está sendo repetido várias vezes em um mesmo registro

#### Define
pecorrer os registros, separar os urls do expanded_urls, retirar os repetidos e atualiizar o registro

#### Code

In [ ]:
urls_list = []
for urls in df_tweets_clean.expanded_urls:
    urls_list.append((',').join(list(set(urls.split(',')))))
df_tweets_clean.expanded_urls = urls_list

#### Test

In [ ]:
for urls in df_tweets_clean.expanded_urls:
    print(urls)

#### `df_tweets`: Existem tweets em que o texto começa com RT idicando um retweet

#### Define
retirar os dados relativos a RT

#### Code

In [ ]:
for key,row in df_tweets_clean.iterrows():
    if(row.text.find('RT') == 0):
        df_tweets_clean = df_tweets_clean.drop(key)

#### Test

In [ ]:
for key,row in df_tweets_clean.iterrows():
    if(row.text.find('RT') == 0):
        print(row.text)

In [ ]:
df_tweets_clean.info()

#### `df_tweets`: A coluna timestamp não está representando um valor timestamp

#### Define
transformar os valores da coluna para timestamp

#### Code

In [ ]:
df_tweets_clean.timestamp = df_tweets_clean.timestamp.astype('datetime64[ns]').astype(np.int64)

#### Test

In [ ]:
df_tweets_clean.timestamp

#### `df_tweets_data`:Registro faltantes(1457 de 2356)

#### Define
adicionar as linhas faltantes com informações nulas

#### Code

In [ ]:
df_tweets_data_clean = df_tweets_data.copy()
for tweet_id in df_tweets_clean.tweet_id:
    if(len(df_tweets_data_clean.loc[df_tweets_data_clean.id == tweet_id]) == 0):
        df_tweets_data_clean = df_tweets_data_clean.append({'id':tweet_id,'favorite_count':None,'retweet_count':None},ignore_index=True)

#### Test

In [ ]:
len(df_tweets_data_clean)

#### As colunas retweeted_status_id,retweeted_status_user_id e retweeted_status_timestamp do df_tweets não são úteis já que não  queremos retweets.

#### Define
Retirar as colunas que não são úteis

#### Code

In [ ]:
df_teste = df_tweets_clean.copy()

In [ ]:
df_teste.drop(['retweeted_status_id'], axis=1, inplace=True,errors='ignore')
df_teste.drop(['retweeted_status_user_id'], axis=1, inplace=True,errors='ignore')
df_teste.drop(['retweeted_status_timestamp'], axis=1, inplace=True,errors='ignore')

In [ ]:
df_tweets_clean = df_teste

#### Test

In [ ]:
df_tweets_clean.info()

#### Os dados do dataframe df_tweets_data fazem parte do df_tweets

#### Define
Adicionar em df_tweets dos dados de df_tweets_data para cada id relacionado

#### Code

In [ ]:
df_tweets_data_clean.id = df_tweets_data_clean.id.astype(np.int64)
df_tweets_data_clean = df_tweets_data_clean.rename(columns={'id':'tweet_id'})

In [ ]:
df_teste = df_tweets_data_clean.copy()
df_teste2 = df_tweets_clean.copy()
df_master = pd.merge(df_teste2,df_teste,on='tweet_id',how='left')

#### Test

In [ ]:
df_master

#### As colunas doggo, floofer, pupper e puppo respresentam a mesma informação que é o estagio do cão.

#### Define
Criar uma coluna que sempre vai possuir o valor de uma das colunas, se alguma estiver preenchida. Para isso, 
pecorrer o df e criar uma coluna

#### Code

In [ ]:
stage_dog = []
for key,row in df_master.iterrows():
    stages = {
        0:None,
        1:'doggo',
        2:'floofer',
        3:'pupper',
        4:'puppo',
        5:'multiple'
    }
    select = 0
    if(row.doggo == 'doggo'):
        select = 1
    if(row.floofer == 'floofer'):
        if(select == 0):
            select = 2
        else:
            select = 5
    if(row.pupper == 'pupper'):
        if(select == 0):
            select = 3
        else:
            select = 5
    if(row.puppo == 'puppo'):
        if(select == 0):
            select = 4
        else:
            select = 5
    stage_dog.append(stages[select])
df_master['stage_dog'] = stage_dog

In [ ]:
df_master.drop(['floofer','doggo','pupper','puppo'], axis=1, inplace=True,errors='ignore')

#### Test

In [ ]:
df_master.info()

In [ ]:
df_master.stage_dog.value_counts()

#### O data frame `df_images_predictions` deve ser uma coluna no df_tweets indicando a raça do  cão caso seja possível

#### Define
Verificar qual predição mais correta e criar uma coluna com estas no dataframe df_images_predctions, clonar e deletar o resto das informações de previsão. Fazer o merge com o master

#### Code

In [ ]:
breed = []
df_images_predictions_clean = df_images_predictions.copy()
for key,row in df_images_predictions_clean.iterrows():
    reliability = 0
    dog = None
    if(reliability < row.p1_conf and row.p1_dog):
        dog = row.p1;
        reliability = row.p1_conf
    elif(reliability < row.p2_conf and row.p2_dog):
        dog = row.p2;
        reliability = row.p2_conf
    elif(reliability < row.p3_conf and row.p3_dog):
        dog = row.p3;
        reliability = row.p3_conf
    breed.append(dog)

In [ ]:
df_images_predictions_clean['breed'] = breed;

In [ ]:
df_images_predictions_clean.drop(['p1','p2','p3','p1_conf','p2_conf','p3_conf','p1_dog','p2_dog','p3_dog','jpg_url','img_num'], axis=1, inplace=True,errors='ignore')
df_master = pd.merge(df_master,df_images_predictions_clean,on='tweet_id',how='left')

#### Test

In [ ]:
df_master

## Save

In [ ]:
df_master.to_csv('../Results/twitter_archive_master.csv')

## Analyze

In [ ]:
df_master_clean = df_master.copy()
grade = []
for key,row in df_master_clean.iterrows():
    if(row.rating_numerator != None and row.rating_denominator != None and row.rating_denominator != 0):
        grade.append(row.rating_numerator/row.rating_denominator)
    else:
        grade.append(None)
df_master_clean['grade'] = grade

## Media de favorite para cada estagio de cão

In [ ]:
favorite_stage = {'floofer':[],'doggo':[],'pupper':[],'puppo':[],'multiple':[]}
for key,row in df_master_clean.iterrows():
    if(row.stage_dog != None and row.favorite_count > 0):
        favorite_stage[row.stage_dog].append(row.favorite_count)
print('floofer: '+str(sum(favorite_stage['floofer'])/len(favorite_stage['floofer'])))
print('doggo: '+str(sum(favorite_stage['doggo'])/len(favorite_stage['doggo'])))
print('pupper: '+str(sum(favorite_stage['pupper'])/len(favorite_stage['pupper'])))
print('puppo: '+str(sum(favorite_stage['puppo'])/len(favorite_stage['puppo'])))
print('multiple: '+str(sum(favorite_stage['multiple'])/len(favorite_stage['multiple'])))

#### Insight:
Os cachorros classificados como doggo e puppo são os que conseguem mais favorite

## Quais as raças de cachorro (top 5) que possuem maior quantidade de favorite

In [ ]:
favorite_breed = {}
for key,row in df_master_clean.iterrows():
    if(row.breed != None and row.favorite_count > 0):
        if(not(row.breed in favorite_breed)):
            favorite_breed[row.breed] = []
        favorite_breed[row.breed].append(row.favorite_count)
print(favorite_breed)

In [ ]:
for breed in favorite_breed:
    favorite_breed[breed] = sum(favorite_breed[breed])/len(favorite_breed[breed])

In [ ]:
sorted(favorite_breed.items(),key=itemgetter(1),reverse=True)

#### A partir dos resultados poodemos ver que as raças que renderam mais favorite foram aquelas  mais exoticas

## Quantidade de favorite para uma  determinada nota

In [ ]:
sns.lmplot('grade', 'favorite_count', data=df_master_clean, aspect=2, order=0.1)

In [ ]:
sns.lmplot('grade', 'favorite_count', data=df_master_clean.loc[df_master_clean.grade <2], aspect=2, order=0.1)

#### Retirando as notas muito altas podemos visualizar de forma clara que os tweets que possuem maiores notas tendem a ter mais favorite